In [1]:
import graph_tool.all as gt
import pandas as pd
import numpy as np
import cloudpickle as pickle
import sys
sys.path.append("../trisbm")
from trisbm import trisbm

In [2]:
with open("brca/trisbm/model.pkl", "rb") as file:
    model = pickle.load(file)

In [3]:
#network_file = "mirdip/HUMAN.mirDIP_top90k.Translated.ts"
network_file = "tarbase/HUMAN.TarBase.Translated.tsv"

In [4]:
df_net = pd.read_csv("../regulatory/"+network_file, sep="\t", header=None)
df_net[0]=[s.replace("miR","mir") for s in df_net[0]]
df_conversion = pd.read_csv("miRNA.txt")
df_gene_conversion = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=gd_pub_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", sep="\t").dropna(how="any", axis=0)
df_net = df_net[df_net[0].isin(model.keywords)]
df_net = df_net[df_net[0].isin(df_conversion["miRBase ID"])]
df_net = df_net[df_net[1].isin(df_gene_conversion["Approved symbol"])]
df_gene_conversion = df_gene_conversion.set_index("Approved symbol")

In [5]:
vertexmap = {}
for i,node in enumerate(np.concatenate((model.documents, model.words, model.keywords))):
    vertexmap[node]=i

In [6]:
def get_regulatory_edge():
    for mirna, target in df_net.values:
        gene = df_gene_conversion.at[target, "Ensembl gene ID"]
        if (mirna in model.keywords) and (gene in model.words):
            #print(mirna, target, gene)
            yield (vertexmap[mirna], vertexmap[gene], 1.)
regulatory_edges = get_regulatory_edge()

In [7]:
print(model.g)
model.g.save("tmp.xml.gz") #need to be stored to remove edges later

<Graph object, undirected, with 5378 vertices and 3632009 edges, 2 internal vertex properties, 1 internal edge property, at 0x7f74a1245df0>


In [8]:
model.g.add_edge_list(regulatory_edges, eprops=[model.g.ep["count"]])

In [9]:
print(model.g)

<Graph object, undirected, with 5378 vertices and 3633235 edges, 2 internal vertex properties, 1 internal edge property, at 0x7f74a1245df0>


In [10]:
state = model.state.copy(g=model.g, bs=model.state.get_bs() + [np.zeros(1)] * 2, sampling = True)

S1 = state.entropy()

for i in range(100):
    state.multiflip_mcmc_sweep(niter=10, beta=np.inf)
    
print(model.get_mdl(), S1, state.entropy())

5414191.155795045 5414115.825543952 5365945.977253478


In [11]:
model.state = state
model.g = gt.load_graph("tmp.xml.gz")

In [12]:
import os

In [13]:
# necessary to be compatible with older versions of trisbm
model.nbranches = 1 #not stored in older versions
model.keywords = [model.keywords] # new format
model.groups = {} # should not be cached

In [ ]:
os.chdir("brca")
os.system("rm -rf tarbase && mkdir -p tarbase")
os.chdir("tarbase")
model.save_data()
os.chdir("../../")

Saving level 3


In [ ]:
os.chdir("../../")